In [2]:
pip install yfinance

  Using cached yfinance-0.2.61-py2.py3-none-any.whl.metadata (5.8 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl.metadata (5.5 kB)
  Using cached peewee-3.18.1.tar.gz (3.0 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached curl_cffi-0.11.1-cp39-abi3-win_amd64.whl.metadata (15 kB)
Using cached yfinance-0.2.61-py2.py3-none-any.whl (117 kB)
Using cached curl_cffi-0.11.1-cp39-abi3-win_amd64.whl (1.4 MB)
Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)
  Created wheel for peewee: filename=peewee-3.18.1-py3-none-any.whl size=139098 sha256=28a7e6b6aac0f15d0e23f2a241635e68f3ff7b5ceedfa7c04cf6e5585b247a5c
  Stored in directory: c:\users\james\appdata\local\pip\cache\wheels\1a\57\6a\b

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

ModuleNotFoundError: No module named 'yfinance'

In [ ]:
tickers = ['APPL','GOOGL','AMZN','MSFT','TSLA']
start_date = '2019-01-01'
end_date = '2025-05-25'
data = {ticker: yf.download(ticker, start=start_date, end=end_date) for ticker in tickers}